### Q1.Select the Tool

In [ ]:
# What's the name of the orchestrator you chose?

# Mage

### Q2. Version

In [1]:
!git clone https://github.com/mage-ai/mlops.git
!cd mlops

# Launch Mage and the database service (PostgreSQL):
!./scripts/start.sh

# Open http://localhost:6789 in browser.

fatal: unknown error occurred while reading the configuration files
/bin/bash: ./scripts/start.sh: No such file or directory


In [ ]:
# What's the version of the orchestractor?

# v0.9.73

### Q3.Creating a pipeline

In [ ]:
# create a data loader block named "load"

# paste codes:
"""
import pandas as pd

df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')
"""

In [ ]:
# How many records did we load (the March 2023 Yellow taxi trips data)?

# 3403766 rows x 19 columns

### Q4.Data preparation

In [ ]:
# create a Transformer block named "transform"

# paste codes:
"""
import pandas as pd

df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.dt.total_seconds() / 60

df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
df[categorical] = df[categorical].astype(str)
"""

In [ ]:
# What's the size of the result?

# 3316216 rows x 20 columns

### Q5.Train a model

In [ ]:
# create a Transformer block named "train"

# paste codes:
"""
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

def transform(df, *args, **kwargs):
    # Specify your transformation logic here
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    train_dicts = df[categorical].to_dict(orient='records')

    dv = DictVectorizer()
    x_train = dv.fit_transform(train_dicts)

    target = 'duration'
    y_train = df[target].values

    lr = LinearRegression()
    lr.fit(x_train, y_train)

    print(lr.intercept_)

    return dv, lr
"""

In [ ]:
# What's the intercept of the model?

# 24.776368754137366

### Q6.Register the model

In [ ]:
# stop Mage with Ctrl+C

# follow the instructions to add scripts

# ./scripts/start.sh

# Open http://localhost:6789 in browser.
# Open http://127.0.0.1:5000 in browser.

In [ ]:
# > Mage:
# create a Data exporter block named "save"

# paste codes:
"""
import pickle
import mlflow

mlflow.set_tracking_uri("http://mlflow:5000")
mlflow.set_experiment("nyc-taxi-experiment")

def export_data(data, *args, **kwargs):
    # Specify your data exporting logic here
    dv, lr = data
    with mlflow.start_run():
        with open('dict_vectorizer.bin', 'wb') as f_out:
            pickle.dump(dv, f_out)
        mlflow.log_artifact('dict_vectorizer.bin')

        mlflow.sklearn.log_model(lr, 'model')

    print('OK')
"""

In [ ]:
# Find the logged model, and find MLModel file. What's the size of the model? (model_size_bytes field):

# model_size_bytes: 4534